In [1]:
from __future__ import print_function
import glob
import os
import numpy as np
from PIL import Image
from shutil import copyfile
import sys
import tarfile
import time

# Loat the right urlretrieve based on python version
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve

import zipfile

# Useful for being able to dump images into the Notebook
import IPython.display as D

# Import CNTK and helpers
import cntk as C

In [2]:
isFast = True


In [3]:
C.device.try_set_default_device(C.device.gpu(0))

True

In [4]:
# By default, we store data in the Examples/Image directory under CNTK
# If you're running this _outside_ of CNTK, consider changing this
data_root = os.path.join('..', 'Examples', 'Image')

datasets_path = os.path.join(data_root, 'DataSets')
output_path = os.path.join('.', 'temp', 'Output')

def ensure_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [5]:
!curl -L -O -S https://www.cntk.ai/Models/Caffe_Converted/VGG16_ImageNet_Caffe.model

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  527M  100  527M    0     0  5847k      0  0:01:32  0:01:32 --:--:-- 6327k


In [6]:
base_model_file="VGG16_ImageNet_Caffe.model"

# Print out all layers in the model
print('Loading {} and printing all layers:'.format(base_model_file))
node_outputs = C.logging.get_node_outputs(C.load_model(base_model_file))
for l in node_outputs: print("  {0} {1}".format(l.name, l.shape))

Loading VGG16_ImageNet_Caffe.model and printing all layers:
  prob (1000,)
  prob (1000,)
  fc8 (1000,)
  drop7 (4096,)
  relu7 (4096,)
  fc7 (4096,)
  drop6 (4096,)
  relu6 (4096,)
  fc6 (4096,)
  pool5 (512, 7, 7)
  relu5_3 (512, 14, 14)
  conv5_3 (512, 14, 14)
  relu5_2 (512, 14, 14)
  conv5_2 (512, 14, 14)
  relu5_1 (512, 14, 14)
  conv5_1 (512, 14, 14)
  pool4 (512, 14, 14)
  relu4_3 (512, 28, 28)
  conv4_3 (512, 28, 28)
  relu4_2 (512, 28, 28)
  conv4_2 (512, 28, 28)
  relu4_1 (512, 28, 28)
  conv4_1 (512, 28, 28)
  pool3 (256, 28, 28)
  relu3_3 (256, 56, 56)
  conv3_3 (256, 56, 56)
  relu3_2 (256, 56, 56)
  conv3_2 (256, 56, 56)
  relu3_1 (256, 56, 56)
  conv3_1 (256, 56, 56)
  pool2 (128, 56, 56)
  relu2_2 (128, 112, 112)
  conv2_2 (128, 112, 112)
  relu2_1 (128, 112, 112)
  conv2_1 (128, 112, 112)
  pool1 (64, 112, 112)
  relu1_2 (64, 224, 224)
  conv1_2 (64, 224, 224)
  relu1_1 (64, 224, 224)
  conv1_1 (64, 224, 224)


In [7]:
C.load_model(base_model_file)

Composite(Combine): Input('data', [#], [3 x 224 x 224]) -> Output('prob', [#], [1000])

In [8]:
# define base model location and characteristics
base_model_file="VGG16_ImageNet_Caffe.model"
base_model = {
    'model_file': base_model_file,
    'feature_node_name': 'data',
    'last_hidden_node_name': 'drop7',
    # Channel Depth x Height x Width
    'image_dims': (3, 224, 224)
}

In [9]:
import cntk.io.transforms as xforms
ensure_exists(output_path)
np.random.seed(123)

# Creates a minibatch source for training or testing
def create_mb_source(map_file, image_dims, num_classes, randomize=True):
    transforms = [xforms.scale(width=image_dims[2], height=image_dims[1], channels=image_dims[0], interpolations='linear')]
    return C.io.MinibatchSource(C.io.ImageDeserializer(map_file, C.io.StreamDefs(
            features=C.io.StreamDef(field='image', transforms=transforms),
            labels=C.io.StreamDef(field='label', shape=num_classes))),
            randomize=randomize)

# Creates the network model for transfer learning
def create_model(model_details, num_classes, input_features, new_prediction_node_name='prediction', freeze=False):
    # Load the pretrained classification net and find nodes
    base_model = C.load_model(model_details['model_file'])
    feature_node = C.logging.find_by_name(base_model, model_details['feature_node_name'])
    last_node = C.logging.find_by_name(base_model, model_details['last_hidden_node_name'])
    # Clone the desired layers with fixed weights
    cloned_layers = C.combine([last_node.owner]).clone(
        C.CloneMethod.freeze if freeze else C.CloneMethod.clone,
        {feature_node: C.placeholder(name='features')})

    # Add new dense layer for class prediction
    feat_norm = input_features #FIXME - C.Constant(114)
    cloned_out = cloned_layers(feat_norm)
    z = C.layers.Dense(num_classes, activation=None, name=new_prediction_node_name) (cloned_out)

    return z

In [10]:
m = C.load_model(base_model_file)
m.output

Output('prob', [#], [1000])

In [11]:
# Trains a transfer learning model
def train_model(model_details, num_classes, train_map_file,
                learning_params, max_images=-1):
    num_epochs = learning_params['max_epochs']
    epoch_size = sum(1 for line in open(train_map_file))
    if max_images > 0:
        epoch_size = min(epoch_size, max_images)
    minibatch_size = learning_params['mb_size']

    # Create the minibatch source and input variables
    minibatch_source = create_mb_source(train_map_file, model_details['image_dims'], num_classes)
    image_input = C.input_variable(model_details['image_dims'])
    label_input = C.input_variable(num_classes)

    # Define mapping from reader streams to network inputs
    input_map = {
        image_input: minibatch_source['features'],
        label_input: minibatch_source['labels']
    }

    # Instantiate the transfer learning model and loss function
    tl_model = create_model(model_details, num_classes, image_input, freeze=learning_params['freeze_weights'])
    ce = C.cross_entropy_with_softmax(tl_model, label_input)
    pe = C.classification_error(tl_model, label_input)

    # Instantiate the trainer object
    lr_schedule = C.learning_rate_schedule(learning_params['lr_per_mb'], unit=C.UnitType.minibatch)
    mm_schedule = C.momentum_schedule(learning_params['momentum_per_mb'])
    learner = C.momentum_sgd(tl_model.parameters, lr_schedule, mm_schedule,
                           l2_regularization_weight=learning_params['l2_reg_weight'])
    trainer = C.Trainer(tl_model, (ce, pe), learner)

    # Get minibatches of images and perform model training
    print("Training transfer learning model for {0} epochs (epoch_size = {1}).".format(num_epochs, epoch_size))
    C.logging.log_number_of_parameters(tl_model)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=num_epochs)
    for epoch in range(num_epochs):       # loop over epochs
        sample_count = 0
        while sample_count < epoch_size:  # loop over minibatches in the epoch
            data = minibatch_source.next_minibatch(min(minibatch_size, epoch_size - sample_count), input_map=input_map)
            trainer.train_minibatch(data)                                    # update model with it
            sample_count += trainer.previous_minibatch_sample_count          # count samples processed so far
            progress_printer.update_with_trainer(trainer, with_metric=True)  # log progress
            if sample_count % (100 * minibatch_size) == 0:
                print ("Processed {0} samples".format(sample_count))

        progress_printer.epoch_summary(with_metric=True)

    return tl_model

In [12]:
# Evaluates a single image using the re-trained model
def eval_single_image(loaded_model, image_path, image_dims):
    # load and format image (resize, RGB -> BGR, CHW -> HWC)
    try:
        img = Image.open(image_path)

        if image_path.endswith("png"):
            temp = Image.new("RGB", img.size, (255, 255, 255))
            temp.paste(img, img)
            img = temp
        resized = img.resize((image_dims[2], image_dims[1]), Image.ANTIALIAS)
        bgr_image = np.asarray(resized, dtype=np.float32)[..., [2, 1, 0]]
        hwc_format = np.ascontiguousarray(np.rollaxis(bgr_image, 2))

        # compute model output
        arguments = {loaded_model.arguments[0]: [hwc_format]}
        output = loaded_model.eval(arguments)

        # return softmax probabilities
        sm = C.softmax(output[0])
        return sm.eval()
    except FileNotFoundError:
        print("Could not open (skipping file): ", image_path)
        return ['None']



# Evaluates an image set using the provided model
def eval_test_images(loaded_model, output_file, test_map_file, image_dims, max_images=-1, column_offset=0):
    num_images = sum(1 for line in open(test_map_file))
    if max_images > 0:
        num_images = min(num_images, max_images)
    if isFast:
        num_images = min(num_images, 300) #We will run through fewer images for test run

    print("Evaluating model output node '{0}' for {1} images.".format('prediction', num_images))

    pred_count = 0
    correct_count = 0
    np.seterr(over='raise')
    with open(output_file, 'wb') as results_file:
        with open(test_map_file, "r") as input_file:
            for line in input_file:
                tokens = line.rstrip().split('\t')
                img_file = tokens[0 + column_offset]
                probs = eval_single_image(loaded_model, img_file, image_dims)

                if probs[0]=='None':
                    print("Eval not possible: ", img_file)
                    continue

                pred_count += 1
                true_label = int(tokens[1 + column_offset])
                predicted_label = np.argmax(probs)
                if predicted_label == true_label:
                    correct_count += 1
                else:
                    print("Wrong prediction for: {0}: real {1}, predicted {2}".format(img_file, true_label, predicted_label))

                np.savetxt(results_file, probs[np.newaxis], fmt="%.3f")
                if pred_count % 100 == 0:
                    print("Processed {0} samples ({1:.2%} correct)".format(pred_count,
                                                                           (float(correct_count) / pred_count)))
                if pred_count >= num_images:
                    break
    print ("{0} of {1} prediction were correct".format(correct_count, pred_count))
    return correct_count, pred_count, (float(correct_count) / pred_count)


In [13]:
force_retraining = True

max_training_epochs = 5 if isFast else 20

learning_params = {
    'max_epochs': max_training_epochs,
    'mb_size': 50,
    'lr_per_mb': [0.2]*10 + [0.1],
    'momentum_per_mb': 0.9,
    'l2_reg_weight': 0.0005,
    'freeze_weights': True
}


In [14]:
# Set python version variable
python_version = sys.version_info.major

def create_map_file_from_folder(root_folder, class_mapping, include_unknown=False, valid_extensions=['.jpg', '.jpeg', '.png']):
    map_file_name = os.path.join(root_folder, "map.txt")

    map_file = None

    if python_version == 3:
        map_file = open(map_file_name , 'w', encoding='utf-8')
    else:
        map_file = open(map_file_name , 'w')

    for class_id in range(0, len(class_mapping)):
        folder = os.path.join(root_folder, class_mapping[class_id])
        if os.path.exists(folder):
            for entry in os.listdir(folder):
                filename = os.path.abspath(os.path.join(folder, entry))
                if os.path.isfile(filename) and os.path.splitext(filename)[1].lower() in valid_extensions:
                    try:
                        map_file.write("{0}\t{1}\n".format(filename, class_id))
                    except UnicodeEncodeError:
                        continue

    if include_unknown:
        for entry in os.listdir(root_folder):
            filename = os.path.abspath(os.path.join(root_folder, entry))
            if os.path.isfile(filename) and os.path.splitext(filename)[1].lower() in valid_extensions:
                try:
                    map_file.write("{0}\t-1\n".format(filename))
                except UnicodeEncodeError:
                    continue

    map_file.close()

    return map_file_name


def create_class_mapping_from_folder(root_folder):
    classes = []
    for _, directories, _ in os.walk(root_folder):
        for directory in directories:
            classes.append(directory)
    return np.asarray(classes)

In [15]:
drinks_data = {
        'training_folder': '/data/drinks/train',
        'testing_folder': '/data/drinks/valid'
    }

drinks_data['class_mapping'] = create_class_mapping_from_folder(drinks_data['training_folder'])
drinks_data['training_map'] = create_map_file_from_folder(drinks_data['training_folder'], drinks_data['class_mapping'])
drinks_data['testing_map'] = create_map_file_from_folder(drinks_data['testing_folder'], drinks_data['class_mapping'])

In [16]:
drinks_model = {
    'model_file': os.path.join(output_path, 'DrinksTransferLearning-VGG.model'),
    'results_file': os.path.join(output_path, 'DrinksPredictions.txt'),
    'num_classes': len(drinks_data['class_mapping'])
}

In [17]:

if os.path.exists(drinks_model['model_file']) and not force_retraining:
    print("Loading existing model from %s" % drinks_model['model_file'])
    trained_model = C.load_model(drinks_model['model_file'])
else:
    trained_model = train_model(base_model,
                                drinks_model['num_classes'], drinks_data['training_map'],
                                learning_params)
    trained_model.save(drinks_model['model_file'])
    print("Stored trained model at %s" % drinks_model['model_file'])

Training transfer learning model for 5 epochs (epoch_size = 570).
Training 12291 parameters in 2 parameter tensors.
Finished Epoch[1 of 5]: [Training] loss = 3.989726 * 570, metric = 27.89% * 570 19.557s ( 29.1 samples/s);
Finished Epoch[2 of 5]: [Training] loss = 0.966719 * 570, metric = 3.16% * 570 7.190s ( 79.3 samples/s);
Finished Epoch[3 of 5]: [Training] loss = 0.317903 * 570, metric = 1.58% * 570 7.118s ( 80.1 samples/s);
Finished Epoch[4 of 5]: [Training] loss = 0.037983 * 570, metric = 0.35% * 570 7.116s ( 80.1 samples/s);
Finished Epoch[5 of 5]: [Training] loss = 0.000288 * 570, metric = 0.00% * 570 7.246s ( 78.7 samples/s);
Stored trained model at ./temp/Output/DrinksTransferLearning-VGG.model


In [18]:
# Evaluate the test set
predict_correct, predict_total, predict_accuracy = \
   eval_test_images(trained_model, drinks_model['results_file'], drinks_data['testing_map'], base_model['image_dims'])
print("Done. Wrote output to %s" % drinks_model['results_file'])


Evaluating model output node 'prediction' for 141 images.
Wrong prediction for: /data/drinks/valid/valser/img014.jpg: real 1, predicted 0
Processed 100 samples (99.00% correct)
140 of 141 prediction were correct
Done. Wrote output to ./temp/Output/DrinksPredictions.txt
